In [20]:
#imports various libraries
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import numpy as np
import pickle

from collections import Counter
from nltk import ngrams
import re
import copy 
import matplotlib.pyplot as plt



/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


# Analysing All URLs

In [124]:

with open('../../Ignore/SCMP_All_URLs.pkl', 'rb') as f:
    df = pickle.load(f)
    
df

,urls,date
0,https://www.scmp.com/news/world/united-states-canada/article/3109887/after-assailing-fox-news-calling-arizona-bidens,2020-11-14 15:39:00+00:00
1,https://www.scmp.com/news/world/europe/article/3109886/coronavirus-europe-further-tightens-curbs-stem-second-wave-global,2020-11-14 14:30:00+00:00
2,https://www.scmp.com/comment/opinion/article/3109884/instilling-patriotism-young-needs-more-rituals-and-indoctrination,2020-11-14 13:24:00+00:00
3,https://www.scmp.com/comment/opinion/article/3109883/ending-hong-kong-sanctions-would-be-good-start-biden,2020-11-14 13:24:00+00:00
4,https://www.scmp.com/sport/racing/article/3109882/forte-justifies-expensive-price-tag-impressive-debut-win-sha-tin,2020-11-14 14:11:00+00:00
...,...,...
1458116,https://www.scmp.com/tech/policy/article/3099673/white-house-will-crack-down-more-chinese-apps-after-tiktok-wechat-trump,2020-09-05 08:23:00+00:00
1458150,https://www.scmp.com/news/world/united-states-canada/article/3099635/coronavirus-us-cases-top-6-million-johns-hopkins,2020-08-31 22:49:00+00:00
1458151,https://www.scmp.com/week-asia/politics/article/3099634/us-india-dialogue-october-china-and-defence-cooperation-will-be,2020-08-31 23:49:00+00:00
1458170,https://www.scmp.com/economy/china-economy/article/3099575/us-china-style-trade-wars-could-proliferate-without-wto,2020-09-02 01:14:00+00:00


In [122]:
links_text = ' '.join(list(df.urls))
links_text = links_text.replace("https://www.scmp.com", " ")
links_text = links_text.replace("/", " ")

ngram_counts = Counter(ngrams(links_text.split(), 1))


In [298]:
ngram_counts.most_common(10)

[(('article',), 1166800),
 (('news',), 176934),
 (('hong-kong',), 63334),
 (('world',), 50692),
 (('china',), 45359),
 (('business',), 43732),
 (('sport',), 40380),
 (('magazines',), 37193),
 (('comment',), 32690),
 (('asia',), 28991)]

In [140]:
#just got the tags manually
contain_china = []
contain_hk = []

for link in df.urls:
    #contain_china
    if '/china/' in link or '/china-digest/' in link or '/china-insider/' in link or '/hong-kong-china/' in link :
        contain_china.append(1)
    else:
        contain_china.append(0)

    #contain_hk
    if '/hong-kong/' in link or '/hong-kong-china/' in link:
        contain_hk.append(1)
    else:
        contain_hk.append(0)

df['China_tag'] = contain_china
df['HK_tag'] = contain_hk

tag_df = df.loc[:, df.columns.intersection(['China_tag','HK_tag'] )]

tag_df = tag_df.groupby(df.date.dt.year).agg(['sum'])
tag_df["ratio"] = tag_df[('China_tag', 'sum')] /  tag_df[('HK_tag', 'sum')]
tag_df

,China_tag,HK_tag,ratio
,sum,sum,
date,,,
1992,0,0,NaN
1993,0,0,NaN
1994,0,1,0.000000
1995,0,3,0.000000
1996,0,1,0.000000
1997,1,0,inf
1998,0,0,NaN
1999,0,0,NaN


# Load Complete News Dataset

In [79]:
#load dataset from scrap notebook
with open('../Pickles/SCMP_News_Full.pkl', 'rb') as f:
    df = pickle.load(f)
    
#for getting china articles
china_df = copy.deepcopy(df)

df.tail()


,Title,Author,Summary,Topic,Link,Date
92246,"Hong Kong police inspector arrested on suspicion of stealing colleagues’ tactical helmets, abalone",Christy Leung,The disappearance of the tactical gear and canned seafood from the offices of the public relations bureau prompted the man’s colleagues to file a complaint.,"Hong Kong police,Crime in Hong Kong",https://www.scmp.com/news/hong-kong/law-and-crime/article/3109872/hong-kong-police-inspector-arrested-suspicion-stealing,2020-11-14 15:28:00+00:00
92247,"Mandatory Covid-19 testing for ‘high-risk’ groups in Hong Kong gets nod, those who ignore orders face jail, fines","Elizabeth Cheung,Gigi Choy,Chris Lau",Other new measures include reintroduction of four-person cap on tables at eateries.,"Coronavirus pandemic,Coronavirus pandemic: All stories,Coronavirus Hong Kong,Hong Kong social distancing,Executive Council of Hong Kong",https://www.scmp.com/news/hong-kong/health-environment/article/3109853/coronavirus-hong-kongs-executive-council-signs,2020-11-14 15:30:00+00:00
92248,Pope Francis said to be in final stage of selecting new Hong Kong bishop,Mimi Lau,"Pontiff has three nominations to consider after an earlier announcement was postponed, according to informed source.","China-Vatican relations,Catholic Church,Hong Kong national security law (NSL),Pope Francis",https://www.scmp.com/news/china/diplomacy/article/3109825/pope-francis-said-be-final-stage-selecting-new-hong-kong,2020-11-14 15:30:00+00:00
92249,"Hong Kong’s controversial national security law tip line gets 10,000 messages in first week of operation",Christy Leung,"By 8am on Thursday, the multi-platform project had gathered the equivalent of 1,400 piece of information a day for the force’s new national security unit.","Hong Kong national security law (NSL),Hong Kong police,Hong Kong protests",https://www.scmp.com/news/hong-kong/law-and-crime/article/3109867/hong-kongs-controversial-national-security-law-tip,2020-11-14 15:47:00+00:00
92250,"Mandatory Covid-19 testing for ‘high-risk’ groups in Hong Kong gets nod, those who ignore orders face jail, fines","Elizabeth Cheung,Gigi Choy,Chris Lau",Other new measures include reintroduction of four-person cap on tables at eateries.,"Coronavirus pandemic,Coronavirus pandemic: All stories,Coronavirus Hong Kong,Hong Kong social distancing,Executive Council of Hong Kong",https://www.scmp.com/news/hong-kong/health-environment/article/3109853/coronavirus-hong-kongs-executive-council-signs,2020-11-14 16:03:00+00:00


# News Articles By Date

In [80]:
years = df.groupby(df.Date.dt.year).agg(['count'])
years[(   'Date', 'count')]

Date
2012    2894 
2013    10730
2014    9089 
2015    10150
2016    12260
2017    12407
2018    13926
2019    11245
2020    9550 
Name: (Date, count), dtype: int64

In [81]:
months = df.groupby([df.Date.dt.year, df.Date.dt.month]).agg(['count'])
#dict(months[(   'Date', 'count')])

# Tag Frequencies

In [82]:
links_text = ' '.join(list(df.Link))
links_text_clean1 = links_text.replace("https://www.scmp.com/news", " ")

In [83]:
tags = set(re.findall(r'  (.*?)article', links_text_clean1))

In [84]:
unique_tags = []
tags_list = list(tags)[1:]
for tag in tags_list:
    u = re.findall(r'/(.*?)/', tag)
    unique_tags.extend(u)

In [85]:
'''
 #contain_southEastAsia
 '/south-east-asia/',
 '/southeast-asia/',
 
 #contain_diplomacy
 '/diplomacy-defence/',
 '/diplomacy/',
 
 #contain_politics
 '/policies-politics/',
 '/poliitics/',
 '/politics-policy/',
 '/politics/',
 
 #contain_economy
 '/economy/',
 '/china-economy/',
  /hong-kong-economy/,
  
  #contain_education
 '/education-community/',
 '/education/',
 
  #contain_lawCrime
  '/hong-kong-law-and-crime/',
  '/law-and-crime/',
  '/law-crime/',
 
 #contain_community
  '/community/',
  '/education-community/',
  
 #contain_environment
 '/environment/',
 '/health-environment/',
 
 #contain_society
 '/social-issues/',
 '/society/',
 
 
 #contain_americas
 '/united-states-canada/',
 '/united-states/',
 '/americas/',
 '/us/'}
 
# 
 

'''
tags


{'/',
 '/asia/',
 '/asia/australasia/',
 '/asia/diplomacy/',
 '/asia/east-asia/',
 '/asia/south-asia/',
 '/asia/south-east-asia/',
 '/asia/southeast-asia/',
 '/china-digest/',
 '/china-insider/',
 '/china/',
 '/china/china-economy/',
 '/china/diplomacy-defence/',
 '/china/diplomacy/',
 '/china/economy/',
 '/china/military/',
 '/china/money-wealth/',
 '/china/policies-politics/',
 '/china/poliitics/',
 '/china/politics-policy/',
 '/china/politics/',
 '/china/science/',
 '/china/society/',
 '/hong-kong/',
 '/hong-kong/community/',
 '/hong-kong/economy/',
 '/hong-kong/education-community/',
 '/hong-kong/education/',
 '/hong-kong/environment/',
 '/hong-kong/health-environment/',
 '/hong-kong/hong-kong-economy/',
 '/hong-kong/hong-kong-law-and-crime/',
 '/hong-kong/law-and-crime/',
 '/hong-kong/law-crime/',
 '/hong-kong/media/',
 '/hong-kong/politics/',
 '/hong-kong/social-issues/',
 '/hong-kong/society/',
 '/hong-kong/transport/',
 '/world/',
 '/world/africa/',
 '/world/americas/',
 '/worl

In [86]:
unique_tags = ["/asia/", "/australasia/", "/east-asia/" ,"/military/"
 , "/money-wealth/" , "/science/", "/transport/", "/media/" , "/world/"
 , "/africa/", "/europe/","/middle-east/","/russia-central-asia/", "/south-asia/"]

tag_dict = {new_list: [] for new_list in unique_tags} 

for key in tag_dict:
    for link in df.Link:
        if key in link:
            tag_dict[key].append(1)
        else:
            tag_dict[key].append(0)
            
col_names = {'/asia/':'Asia', '/australasia/':'Australasia', '/east-asia/':'EastAsia',
      '/military/':'Military', '/money-wealth/':'MoneyWealth', '/science/':'Science',
      '/transport/':'Transport', '/media/':'Media', '/world/':'World',
      '/africa/':'Africa', '/europe/':'Europe', '/middle-east/':'MiddleEast',
     '/russia-central-asia/':'RussiaCentralAsia', '/south-asia/':'SouthAsia'}

tag_dict_list = dict((col_names[key], value) for (key, value) in tag_dict.items())

In [87]:
b = pd.concat({k: pd.Series(v) for k, v in tag_dict_list.items()}, axis=1)
b

,Asia,Australasia,EastAsia,Military,MoneyWealth,Science,Transport,Media,World,Africa,Europe,MiddleEast,RussiaCentralAsia,SouthAsia
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92246,0,0,0,0,0,0,0,0,0,0,0,0,0,0
92247,0,0,0,0,0,0,0,0,0,0,0,0,0,0
92248,0,0,0,0,0,0,0,0,0,0,0,0,0,0
92249,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [88]:
#just got the tags manually
contain_china = []
contain_hk = []
contain_southEastAsia = []
contain_diplomacy = []
contain_politics = []
contain_economy = []
contain_education = []
contain_lawCrime = []
contain_community = []
contain_environment = []
contain_society = []
contain_americas = []

for link in df.Link:
    #contain_china
    if '/china/' in link or '/china-digest/' in link or '/china-insider/' in link :
        contain_china.append(1)
    else:
        contain_china.append(0)
    
    #contain_hk
    if '/hong-kong/' in link:
        contain_hk.append(1)
    else:
        contain_hk.append(0)

    #contain_southEastAsia
    if '/south-east-asia/' in link or '/southeast-asia/' in link:
        contain_southEastAsia.append(1)
    else:
        contain_southEastAsia.append(0)
        
    #contain_diplomacy 
    if '/diplomacy-defence/' in link or '/diplomacy/' in link:
        contain_diplomacy.append(1)
    else:
        contain_diplomacy.append(0)

    #contain_politics 
    if '/policies-politics/' in link or '/politics/' in link or '/politics-policy/' in link or '/poliitics/' in link:
        contain_politics.append(1)
    else:
        contain_politics.append(0)
        
    #contain_economy 
    if '/economy/' in link or '/china-economy/' in link or '/hong-kong-economy/' in link:
        contain_economy.append(1)
    else:
        contain_economy.append(0)

    #contain_education
    if '/education-community/' in link or '/education/' in link:
        contain_education.append(1)
    else:
        contain_education.append(0)
        
    #contain_lawCrime 
    if '/hong-kong-law-and-crime/' in link or '/law-and-crime/' in link or '/law-crime/' in link :
        contain_lawCrime.append(1)
    else:
        contain_lawCrime.append(0)
        
    #contain_community 
    if '/community/' in link or '/education-community/' in link:
        contain_community.append(1)
    else:
        contain_community.append(0)
        
    #contain_environment 
    if '/environment/' in link or '/health-environment/' in link:
        contain_environment.append(1)
    else:
        contain_environment.append(0)
        
    #contain_society 
    if '/social-issues/' in link or '/society/' in link:
        contain_society.append(1)
    else:
        contain_society.append(0)
        
    #contain_americas 
    if '/united-states-canada/' in link or '/americas/' in link or '/united-states/' in link or '/us/' in link:
        contain_americas.append(1)
    else:
        contain_americas.append(0)
    

In [89]:
china_df['China'] = contain_china

df['China'] = contain_china
df['HongKong'] = contain_hk
df['SouthEastAsia'] = contain_southEastAsia
df['Diplomacy'] = contain_diplomacy
df['Politics'] = contain_politics
df['Economy'] = contain_economy
df['Education'] = contain_education
df['LawAndCrime'] = contain_lawCrime
df['Community'] = contain_community
df['Environment'] = contain_environment
df['Society'] = contain_society
df['Americas'] = contain_americas

df = pd.concat([df, b], axis=1, sort=False)


In [90]:
df = df.drop(columns = ["Title","Author","Summary", "Topic", "Link"])
df

,Date,China,HongKong,SouthEastAsia,Diplomacy,Politics,Economy,Education,LawAndCrime,Community,...,MoneyWealth,Science,Transport,Media,World,Africa,Europe,MiddleEast,RussiaCentralAsia,SouthAsia
0,2012-08-12 13:59:00+00:00,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2012-08-12 14:17:00+00:00,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2012-08-12 15:57:00+00:00,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2012-08-12 16:59:00+00:00,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2012-08-12 23:13:00+00:00,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92246,2020-11-14 15:28:00+00:00,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
92247,2020-11-14 15:30:00+00:00,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
92248,2020-11-14 15:30:00+00:00,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
92249,2020-11-14 15:47:00+00:00,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [91]:
tag_df = df.groupby([df.Date.dt.year, df.Date.dt.month]).agg(['sum'])
tag_df.columns = tag_df.columns.droplevel(1)


In [92]:
new_index = []
for i in tag_df.index.to_flat_index():
    i = str(i)[1:-1]
    new_index.append(i)
tag_df.index = new_index
tag_df

,China,HongKong,SouthEastAsia,Diplomacy,Politics,Economy,Education,LawAndCrime,Community,Environment,...,MoneyWealth,Science,Transport,Media,World,Africa,Europe,MiddleEast,RussiaCentralAsia,SouthAsia
"2012, 8",82,327,0,0,0,0,0,0,0,0,...,0,0,0,0,7,0,0,0,0,0
"2012, 9",155,527,0,0,0,0,0,0,0,0,...,0,0,0,0,25,0,0,0,0,0
"2012, 10",223,539,0,0,0,0,0,0,0,0,...,0,0,0,0,55,0,0,0,0,0
"2012, 11",7,29,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"2012, 12",252,523,0,0,0,0,0,0,0,0,...,0,0,0,0,45,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"2020, 7",361,481,13,162,184,35,27,104,0,88,...,0,38,13,0,57,4,10,1,1,2
"2020, 8",343,548,7,147,120,66,63,131,0,127,...,1,35,15,0,38,2,11,2,0,4
"2020, 9",349,496,20,166,119,48,43,146,0,100,...,1,39,19,0,30,0,9,2,2,1
"2020, 10",399,445,13,174,142,52,28,114,0,75,...,0,32,30,0,40,0,14,1,2,0


In [93]:
tag_df.T.to_csv("../../../Capstone_Idyll/data/tagged_observable.csv")

# Get Ratio

In [94]:
ratio_df = df.groupby([df.Date.dt.year]).agg(['sum'])
ratio_df.columns = ratio_df.columns.droplevel(1)

ratio = ratio_df['China'] /  ratio_df['HongKong']

In [95]:
ratio_df = pd.DataFrame.from_dict(dict(ratio), orient='index',
                       columns=[ 'Ratio'])

ratio_df.reset_index(inplace=True)
ratio_df = ratio_df.rename(columns={'index':'Date'})
ratio_df

,Date,Ratio
0,2012,0.369666
1,2013,0.581380
2,2014,0.578819
3,2015,0.457020
4,2016,0.525302
5,2017,0.736247
6,2018,0.687087
7,2019,0.719660
8,2020,0.765895


In [96]:
ratio_df.to_csv("../../../Capstone_Idyll/data/ratio_data.csv", index=False)

# Get China Articles

In [334]:
china_df = china_df[china_df.China != 0]
china_df = china_df.drop(columns = ["Topic", "China", "Summary"])


,Title,Author,Link,Date
10,China's team praised as London Olympics close,Chan Kin Wa,https://www.scmp.com/news/china/article/1008790/chinas-team-praised-london-olympics-close,2012-08-13 00:57:00+00:00
12,Billionaire Chen Guangbiao promotes plan for bottled air by smashing e-bicycles,Laura Zhou,https://www.scmp.com/news/china/article/1008791/billionaire-chen-guangbiao-promotes-plan-bottled-air-smashing-e-bicycles,2012-08-13 01:00:00+00:00
13,Taiwanese legislators threaten to block cross-strait investment protection pact,Lawrence Chung,https://www.scmp.com/news/china/article/1008777/taiwanese-legislators-threaten-block-cross-strait-investment-protection,2012-08-13 01:04:00+00:00
18,Outspoken playwright Sha urges Hongkongers to keep democratic voice strong,LOUISE HO,https://www.scmp.com/news/china/article/1008774/outspoken-playwright-sha-urges-hongkongers-keep-democratic-voice-strong,2012-08-13 02:55:00+00:00
19,Army joins police in hunt for killer in Chongqing,Mimi Lau,https://www.scmp.com/news/china/article/1008779/army-joins-police-hunt-killer-chongqing,2012-08-13 04:12:00+00:00
...,...,...,...,...
92240,TikTok gets 15-day extension as US sale deadline is pushed to November 27,Jodi Xu Klein,https://www.scmp.com/news/china/article/3109834/tiktok-gets-15-day-extension-us-sale-deadline-pushed-november-27,2020-11-14 14:49:00+00:00
92241,Does Taiwan’s caution on ‘military training’ show US support has limits?,Lawrence Chung,https://www.scmp.com/news/china/military/article/3109789/does-taiwans-caution-military-training-show-us-support-has,2020-11-14 14:49:00+00:00
92243,South China Sea: will Joe Biden take a more cautious approach in the disputed waters?,Laura Zhou,https://www.scmp.com/news/china/diplomacy/article/3109773/south-china-sea-will-joe-biden-take-more-cautious-approach,2020-11-14 15:21:00+00:00
92245,A harder US line? Potential Pentagon chief floated idea to sink China fleet in 72 hours,Liu Zhen,https://www.scmp.com/news/china/military/article/3109852/harder-us-line-potential-pentagon-chief-floated-idea-sink-china,2020-11-14 15:21:00+00:00


In [2]:
china_df.to_csv("../../../Capstone_Idyll/data/china_colab.csv",index = False)

NameError: name 'china_df' is not defined

# China Sentiment

In [69]:
colab_df = pd.read_csv('../Pickles/china_df.csv')
colab_df.Date = pd.to_datetime(colab_df.Date)
colab_df = colab_df[~colab_df.Title.str.contains("China Digest")]
colab_df.head(5)

,Unnamed: 0,Link,Title,Author,Date,Prediction,Negative_Score,Positive_Score
0,0,https://www.scmp.com/news/china/article/1008790/chinas-team-praised-london-olympics-close,China's team praised as London Olympics close,Chan Kin Wa,2012-08-13 00:57:00+00:00,1,0.001294,0.998706
1,1,https://www.scmp.com/news/china/article/1008791/billionaire-chen-guangbiao-promotes-plan-bottled-air-smashing-e-bicycles,Billionaire Chen Guangbiao promotes plan for bottled air by smashing e-bicycles,Laura Zhou,2012-08-13 01:00:00+00:00,1,0.394052,0.605948
2,2,https://www.scmp.com/news/china/article/1008777/taiwanese-legislators-threaten-block-cross-strait-investment-protection,Taiwanese legislators threaten to block cross-strait investment protection pact,Lawrence Chung,2012-08-13 01:04:00+00:00,0,0.858571,0.141430
3,3,https://www.scmp.com/news/china/article/1008774/outspoken-playwright-sha-urges-hongkongers-keep-democratic-voice-strong,Outspoken playwright Sha urges Hongkongers to keep democratic voice strong,LOUISE HO,2012-08-13 02:55:00+00:00,1,0.000439,0.999561
4,4,https://www.scmp.com/news/china/article/1008779/army-joins-police-hunt-killer-chongqing,Army joins police in hunt for killer in Chongqing,Mimi Lau,2012-08-13 04:12:00+00:00,1,0.145623,0.854377


In [70]:
colab_df = colab_df.drop(columns = ["Unnamed: 0", "Title",'Author',"Link", "Negative_Score","Prediction"])
colab_df = colab_df.groupby([colab_df.Date.dt.year, colab_df.Date.dt.month]).agg(['mean'])
#time_df = time_df.groupby([time_df.Date.dt.year]).agg(['mean'])

new_index = []
for i in colab_df.index.to_flat_index():
    i = str(i)[1:-1]
    new_index.append(i)
colab_df.index = new_index
colab_df.reset_index(inplace=True)
colab_df.columns = colab_df.columns.droplevel(1)

colab_df= colab_df.rename(columns={'index':'Date'})

In [72]:
colab_df.to_csv("../../../Capstone_Idyll/data/china_sentiment_data.csv", index=False)